In [76]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


In [77]:
import os

In [78]:
os.getcwd()

'E:\\DataSpell\\dsProject\\semi_project-main'

In [ ]:
Preprocessing().

In [87]:
preprocessing_db = Preprocessing()
preprocessing_db.drop_columns()
# 연속형 데이터 대체
preprocessing_db.numerical_columns_replace()
# 범주형 데이터 대체ㅐ
preprocessing_db.category_columns_replace()


In [96]:
df = preprocessing_db.get_df()

In [102]:
df.head(5)

,ID,Client_Income,Credit_Amount,Loan_Annuity,Accompany_Client,Client_Income_Type,Client_Education,Client_Marital_Status,Client_Gender,Loan_Contract_Type,...,Application_Process_Hour_14.0,Application_Process_Hour_15.0,Application_Process_Hour_16.0,Application_Process_Hour_17.0,Application_Process_Hour_18.0,Application_Process_Hour_19.0,Application_Process_Hour_20.0,Application_Process_Hour_21.0,Application_Process_Hour_22.0,Application_Process_Hour_23.0
0,12142509,6750.0,61190.55,3416.85,Alone,Commercial,Secondary,M,Male,CL,...,0,0,0,1,0,0,0,0,0,0
1,12138936,20250.0,15282.00,1826.55,Alone,Service,Graduation,M,Male,CL,...,0,0,0,0,0,0,0,0,0,0
2,12181264,18000.0,59527.35,2788.20,Alone,Service,Graduation dropout,W,Male,CL,...,0,0,0,0,0,0,0,0,0,0
3,12188929,15750.0,53870.40,2295.45,Alone,Retired,Secondary,M,Male,CL,...,0,1,0,0,0,0,0,0,0,0
4,12133385,33750.0,133988.40,3547.35,Alone,Commercial,Secondary,M,Female,CL,...,0,0,0,0,0,0,0,0,0,0


In [111]:
df.Client_Income_Type.head(5)

0    Commercial
1       Service
2       Service
3       Retired
4    Commercial
Name: Client_Income_Type, dtype: object

In [121]:
cat_col = df.columns[df.dtypes=='object'].to_list()

In [122]:
num_col =df.columns[df.dtypes=='float64'].to_list()

In [124]:
target_df = preprocessing_db.get_target_df()


In [125]:
standard_scaler = StandardScaler()
onehot_encoder = OneHotEncoder()
prep_processor = ColumnTransformer(
     [('standard',standard_scaler,num_col),
     ('onehot',onehot_encoder, cat_col),
      
     ],remainder='passthrough'
)


In [126]:
train_x, test_x, train_y, test_y = train_test_split(df, target_df, test_size=0.2, random_state=42, stratify=target_df)
# smote = SMOTE(random_state=42)
# X_train_over, y_train_over = smote.fit_sample(train_x, train_y)


In [127]:
prep_processor.fit(train_x)

ColumnTransformer(remainder='passthrough',
                  transformers=[('standard', StandardScaler(),
                                 ['Client_Income', 'Credit_Amount',
                                  'Loan_Annuity', 'Population_Region_Relative',
                                  'Age_Days', 'Employed_Days',
                                  'Registration_Days', 'ID_Days',
                                  'Score_Source_2', 'Phone_Change',
                                  'Credit_Bureau']),
                                ('onehot', OneHotEncoder(),
                                 ['Accompany_Client', 'Client_Income_Type',
                                  'Client_Education', 'Client_Marital_Status',
                                  'Client_Gender', 'Loan_Contract_Type',
                                  'Client_Housing_Type', 'Client_Occupation',
                                  'Client_Permanent_Match_Tag',
                                  'Client_Contact_Work_Tag'])])

In [128]:
scaled_train_x=prep_processor.transform(train_x)
scaled_test_x=prep_processor.transform(test_x)

In [130]:
!pip install xgboost 

     ---------------------------------------- 0.0/70.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/70.9 MB ? eta -:--:--
     --------------------------------------- 0.0/70.9 MB 660.6 kB/s eta 0:01:48
     --------------------------------------- 0.0/70.9 MB 660.6 kB/s eta 0:01:48
     --------------------------------------- 0.0/70.9 MB 660.6 kB/s eta 0:01:48
     --------------------------------------- 0.1/70.9 MB 328.2 kB/s eta 0:03:36
     --------------------------------------- 0.1/70.9 MB 504.4 kB/s eta 0:02:21
     --------------------------------------- 0.1/70.9 MB 504.4 kB/s eta 0:02:21
     --------------------------------------- 0.1/70.9 MB 423.5 kB/s eta 0:02:48
     --------------------------------------- 0.2/70.9 MB 620.6 kB/s eta 0:01:54
     --------------------------------------- 0.3/70.9 MB 714.4 kB/s eta 0:01:39
     --------------------------------------- 0.3/70.9 MB 774.0 kB/s eta 0:01:32
     --------------------------------------- 0.3/70.9 MB

In [150]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score

In [138]:
xgb_params = {
    'n_estimators' : [50,60,70,80,90,100],
    'max_depth' : [3,4,5,6,7,8,9,10],
    'min_child_weight' :[1,2,3,5,10,15]
}

In [139]:
xgb_model = xgb.XGBClassifier()

In [140]:
gscv_xgb = GridSearchCV(estimator=xgb_model, param_grid=xgb_params,scoring='accuracy', cv=3, n_jobs=-1, verbose=3)

In [141]:
gscv_xgb.fit(scaled_train_x, train_y)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                         'min_child_weight': [1, 2, 3, 5, 10, 15],
                         'n_estimators': [50, 60, 70, 80, 90, 100]},
             scoring='accuracy', verbose=3)

In [142]:
import joblib

In [143]:
joblib.dump(gscv_xgb, 'gscv_xgb.pkl')

['gscv_xgb.pkl']

In [144]:
gscv_xgb.best_score_

0.9215050680078839

In [145]:
gscv_xgb.best_params_

{'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 100}

In [147]:
gscv_xgb.best_estimator_.score(scaled_test_x,test_y)

0.922821270310192

array([0, 0, 0, ..., 0, 0, 0])

In [152]:
recall_score(test_y,gscv_xgb.best_estimator_.predict(scaled_test_x) )

0.07465718638902996

In [153]:
xgb_params2 = {
    'n_estimators' : [80,90,100],
    'max_depth' : [3,4,5,6,7,],
    'min_child_weight' :[1,2,3,5,10,15]
}

In [161]:
gscv_xgb2 = GridSearchCV(estimator=xgb_model, param_grid=xgb_params,scoring='recall', cv=3, n_jobs=-1, verbose=3)

In [164]:
gscv_xgb2.fit(scaled_train_x,train_y)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                         'min_child_weight': [1, 2, 3, 5, 10, 15],
                         'n_estimators': [50, 60, 70, 80, 90, 100]},
             scoring='recall', verbose=3)

In [165]:
joblib.dump(gscv_xgb2, 'gscv_xgb2.pkl')

['gscv_xgb2.pkl']

In [166]:
recall_score(test_y,gscv_xgb2.best_estimator_.predict(scaled_test_x) )

0.07465718638902996

In [167]:
gscv_xgb2.best_estimator_.score(scaled_test_x,test_y)

0.922821270310192